# quick_pp

In [ ]:
import sys

sys.path.append("..")
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import pickle

from quick_pp.database.objects import Project
from quick_pp.database.db_connector import DBConnector

db_conn = DBConnector()

project_name = "MOCK_carbonate"
with db_conn.get_session() as db_session:
    # Load well from saved file
    project = Project(db_session, name=project_name)
    print(project.get_well_names())
    all_data = project.get_all_data()
all_data["CPERM"] = all_data.CORE_PERM
all_data["CPORE"] = all_data.CORE_POR / 100

# Water Saturation Estimation

Water saturation estimation is crucial in petrophysics for several reasons:

1. **Hydrocarbon Volume Calculation**: It helps determine the volume of hydrocarbons in place. Accurate water saturation (Sw) values are essential for calculating the original oil in place (OOIP) and original gas in place (OGIP) volumes¹(https://petroshine.com/fluid-saturation/).
2. **Reservoir Characterization**: Understanding the distribution of water saturation helps in characterizing the reservoir, which is vital for planning production strategies and enhancing recovery¹(https://petroshine.com/fluid-saturation/).
3. **Production Forecasting**: Sw values are used in reservoir models to predict future production and to evaluate the economic viability of the reservoir²(https://www.mdpi.com/2077-1312/9/6/666).

### Methods to Estimate Water Saturation

1. **Resistivity Logs**: This is the most common method, where water saturation is estimated using resistivity measurements from well logs. The Archie equation is often used for clean sands, while modified versions like the Waxman-Smits model are used for shaly sands³(https://petrowiki.spe.org/Water_saturation_determination).
2. **Capillary Pressure Measurements**: Laboratory measurements of capillary pressure and corresponding water saturation provide detailed information about the pore structure and fluid distribution³(https://petrowiki.spe.org/Water_saturation_determination).
3. **Core Analysis**: Direct measurement of water saturation from core samples using techniques like the Dean-Stark method³(https://petrowiki.spe.org/Water_saturation_determination).
4. **Nuclear Magnetic Resonance (NMR)**: NMR logging tools can provide estimates of water saturation by measuring the response of hydrogen nuclei in the formation fluids³(https://petrowiki.spe.org/Water_saturation_determination).

This notebook estimates the water saturation using Archie equation and saturation height function based on the capillary pressure measurement.


***
# Log Derived Water Saturation

Estimation of Rw based on formation water salinity, assuming the depths are already in True Vertical Depth Sub Sea (TVDSS).
The range of Rw used in the original paper is 0.015 to 0.03 ohm.m

Estimation of cementation factor (m) based on pickett plot.

In [ ]:
from ipywidgets import widgets, interact

from quick_pp.saturation import pickett_plot

water_wells = ["HW-5", "HW-7", "HW-8", "HW-9", "HW-28", "HW-31"]
focused_data = all_data[all_data.WELL_NAME.isin(water_wells)].copy()
# Use provided PHIE if estimated PHIT not available
focused_data["PHIT"] = focused_data["PHIT"].fillna(focused_data["PHIE"])

wells = widgets.SelectMultiple(
    options=["All"] + list(focused_data["WELL_NAME"].unique()),
    value=["All"],
    description="Wells:",
)
m = widgets.FloatSlider(value=2, min=1, max=3, step=0.1, readout_format=".1f")
min_rw = widgets.FloatSlider(
    value=0.01, min=0.001, max=0.1, step=0.001, readout_format=".3f"
)
min_depth = widgets.FloatSlider(
    value=focused_data.DEPTH.min(),
    min=focused_data.DEPTH.min(),
    max=focused_data.DEPTH.max() - 10,
    step=0.1,
    readout_format=".1f",
)
max_depth = widgets.FloatSlider(
    value=focused_data.DEPTH.max(),
    min=focused_data.DEPTH.min() + 10,
    max=focused_data.DEPTH.max(),
    step=0.1,
    readout_format=".1f",
)


@interact(wells=wells, m=m, min_rw=min_rw, min_depth=min_depth, max_depth=max_depth)
def param(wells, m, min_rw, min_depth, max_depth):
    if "All" in wells:
        data = focused_data[
            (focused_data.DEPTH >= min_depth) & (focused_data.DEPTH <= max_depth)
        ]
    else:
        data = focused_data[
            (focused_data.WELL_NAME.isin(wells))
            & (focused_data.DEPTH >= min_depth)
            & (focused_data.DEPTH <= max_depth)
        ]
    pickett_plot(
        data["RT"],
        data["PHIT"],
        m=m,
        min_rw=min_rw,
        title=f"Pickett Plot for {wells[0]}",
    )

In [ ]:
from quick_pp.saturation import swirr_xplot

swirr_xplot(all_data.SWT, all_data.PHIT)

In [ ]:
import numpy as np
from quick_pp.saturation import (
    estimate_temperature_gradient,
    estimate_rw_temperature_salinity,
    archie_saturation,
)

hw = all_data[all_data.WELL_NAME == "HW-32"].copy()

# Debug water saturation
water_salinity = 100e3
m = 2

temp_grad = estimate_temperature_gradient(hw["DEPTH"], "imperial")
rw = estimate_rw_temperature_salinity(temp_grad, water_salinity)
# rw = np.full(len(hw), 0.03)

swt = archie_saturation(hw["RT"], rw, hw["PHIT"], m=m, n=2)
# swt = swt.clip(0, 1.1)

fig, axes = plt.subplots(2, 1, figsize=(15, 3), sharex=True)
axes[0].plot(hw["DEPTH"], swt, label="SWT")
axes[0].plot(hw["DEPTH"], hw["SW"], label="SW")
axes[0].plot(hw["DEPTH"], np.ones(len(hw)), color="black", linestyle="--")
axes[0].set_ylim(0, 2)
axes[0].legend()

axes[1].plot(hw["DEPTH"], rw, label="RW")
axes[1].set_yscale("log")
axes[1].legend()
fig.tight_layout()

***
# Saturation Height Function

## Core Data

Explain the data source; measurement techniques

### Define Rock Type

Define the rock type based on FZI cut-offs from previous notebook


In [ ]:
32 * np.cos(np.radians(30))

In [ ]:
from quick_pp.rock_type import calc_fzi, rock_typing, calc_r35, plot_fzi, plot_winland
from quick_pp.core_analysis import (
    fit_j_curve,
    j_xplot,
    leverett_j,
    sw_shf_leverett_j,
    poroperm_xplot,
    pc_xplot,
)

core_data = pd.read_csv(r"data\01_raw\COSTA\HW_core_data_all.csv")
core_data["CPORE"] = core_data["Phi (frac.)"]
core_data["CPERM"] = core_data["K mD"]
core_data["PC"] = core_data["O/B Pc (psia)"]
core_data["PC_RES"] = core_data["O/B Pc (psia)"] * 0.088894  # oil-brine system
core_data["SW"] = core_data["Equiv Brine Sat. (Frac.)"]
core_data["SWN"] = core_data.groupby("Sample")["SW"].transform(
    lambda x: (x - x.min()) / (x.max() - x.min())
)

# Calculate J
ift = 32
theta = 30

core_data["J"] = leverett_j(
    core_data["PC_RES"], ift, theta, core_data["CPERM"], core_data["CPORE"]
)

# Filter data
conditions = (
    (core_data["K mD"] > 0) & (core_data["Class"] == "Good")
    # & (core_data['PC'] <= 40)
)
core_data = core_data[conditions].copy()
core_data.drop_duplicates(subset=["CPORE", "CPERM", "SW"], keep="last", inplace=True)

# Plot J
j_xplot(core_data["SWN"], core_data["J"], core_group=core_data["Sample"], ylim=(0, 15))

***
## Leverett J Method using FZI Rock Types

Define the rock type based on FZI cut-offs from previous notebook

In [ ]:
import json
from quick_pp.rock_type import plot_fzi

# Load FZI cutoffs
with open(rf"data\04_project\{project_name}\outputs\fzi_cutoffs.json", "rb") as file:
    fzi_cutoffs = json.load(file)

# FZI
fzi = calc_fzi(core_data["CPORE"], core_data["CPERM"])
rock_flag = rock_typing(fzi, fzi_cutoffs, higher_is_better=True)
core_data["ROCK_FLAG"] = rock_flag

plot_fzi(
    core_data["CPORE"], core_data["CPERM"], rock_type=rock_flag, cut_offs=fzi_cutoffs
)
print(pd.Series(rock_flag).value_counts().sort_index())

In [ ]:
# Plot PTSD distribution
import numpy as np

from quick_pp.rock_type import estimate_pore_throat

copy_df = core_data.copy()
for sample, data in core_data.groupby("Sample"):
    r = estimate_pore_throat(data["PC"], 32, 30)
    log_r = np.log10(r)
    dsw = np.gradient(data["SW"], log_r)
    copy_df.loc[data.index, "R"] = r
    copy_df.loc[data.index, "LOG_R"] = log_r
    copy_df.loc[data.index, "DSW"] = dsw

fig, axes = plt.subplots(4, 3, figsize=(15, 17))
axes = axes.flatten()
for i in range(11):
    rock = i + 1
    data = copy_df[copy_df.ROCK_FLAG == rock]
    ax = axes[i]
    for sample, sample_data in data.groupby("Sample"):
        ax.plot(
            sample_data["LOG_R"],
            sample_data["DSW"],
            label=f"Sample {sample}",
            zorder=-1,
        )
        color = ax.lines[-1].get_color()
        ax.fill_between(
            sample_data["LOG_R"], sample_data["DSW"], color=color, alpha=0.9
        )
    ax.set_title(f"PRT {int(rock)}")
    ax.set_xlabel("Log Pore Throat Radius (microns)")
    ax.set_xlim(-2, 2)
    ax.set_ylabel("dSw/dLogR")
    ax.legend(loc=2, prop={"size": 5})

# Hide any unused subplots
for j in range(i + 1, len(axes)):
    fig.delaxes(axes[j])

fig.set_facecolor("aliceblue")
plt.tight_layout()

In [ ]:
import matplotlib.pyplot as plt

# Get unique rock flags
unique_rock_flags = sorted(core_data["ROCK_FLAG"].unique())

# Create subplots
fig, axes = plt.subplots(nrows=4, ncols=3, figsize=(15, 20))
axes = axes.flatten()

# Plot Pc vs SW for each rock flag
for i in range(11):
    rock = i + 1
    data = copy_df[copy_df.ROCK_FLAG == rock]
    ax = axes[i]
    data = core_data[core_data["ROCK_FLAG"] == rock]
    for sample, sample_data in data.groupby("Sample"):
        ax.plot(sample_data["SWN"], sample_data["PC_RES"], label=f"Sample {sample}")
    ax.set_ylabel("Pc (psia)")
    ax.set_xlabel("SW (frac)")
    ax.set_ylim(0, 10)
    ax.set_xlim(0, 1)
    ax.set_title(f"RRT {int(rock)}")
    ax.legend()
    ax.grid(True)

# Hide any unused subplots
for j in range(i + 1, len(axes)):
    fig.delaxes(axes[j])

fig.set_facecolor("aliceblue")
plt.tight_layout()
plt.show()

In [ ]:
from ipywidgets import interact, widgets
import plotly.graph_objects as go

from quick_pp.core_analysis import pc_xplot, poroperm_xplot, pc_xplot_plotly

rock_flag_widget = widgets.SelectMultiple(
    options=["All"] + sorted(list(core_data["ROCK_FLAG"].unique())),
    value=["All"],
    description="Rock Flag:",
)


@interact(rock_flag=rock_flag_widget)
def param(rock_flag):
    # Plot all data on poroperm plot
    poroperm_xplot(core_data["CPORE"], core_data["CPERM"])
    data = (
        core_data[core_data.ROCK_FLAG.isin(rock_flag)]
        if any([l for l in rock_flag if l != "All"])
        else core_data
    )

    # Plot filtered data
    poroperm_data = data.drop_duplicates(subset=["CPORE", "CPERM"], keep="last")
    poroperm_xplot(
        poroperm_data["CPORE"],
        poroperm_data["CPERM"],
        core_group=poroperm_data["Sample"],
    )
    plt.show()

    fig = go.Figure()
    for label, temp_df in data.groupby("Sample"):
        fig = pc_xplot_plotly(
            temp_df["SWN"], temp_df["PC_RES"], label=label, ylim=(0, 1.5), fig=fig
        )
    fig.show()


plt.close("all")

#### QC the Pc data

The capillary pressure measurements for each Sample are plotted on a log-log plot.
The data points should fall on a relatively straight line indicating good data quality.

Based 
select the dataset for each rock type
curve fitting

In [ ]:
from ipywidgets import interact, widgets

from quick_pp.core_analysis import fit_j_curve

prt = widgets.Dropdown(
    options=sorted(core_data["ROCK_FLAG"].unique()), description="PRT:"
)

conditions = (core_data["SWN"] > 0.1) & (core_data["SWN"] < 0.9)
filtered_data = core_data[conditions].copy()


@interact(prt=prt)
def param(prt):
    data = filtered_data[filtered_data["ROCK_FLAG"] == prt]
    a, b = fit_j_curve(data["SWN"], data["J"])
    j_xplot(
        data["SWN"],
        data["J"],
        a=a,
        b=b,
        label=f"Sample {sample}: a:{a}, b:{b}",
        core_group=data["Sample"],
        log_log=True,
        ylim=(0, 0.5),
    )

In [ ]:
from sklearn.metrics import root_mean_squared_error

excluded_samples = [
    72,
    77,
    81,
    51,
    38,
    54,
    26,
    66,
    43,
    69,
    55,
    57,
    75,
    # 50, 63, 80,
    22,
    28,
    31,
    34,
    39,
    42,
    44,
    45,
    46,
    52,
    46,
    59,
    61,
    62,
    65,
    25,
    21,
    16,
    24,
    20,
    33,
    13,
    11,
    7,
    6,
]
j_params = {}
for sample, data in filtered_data.groupby("ROCK_FLAG"):
    data = data[~data["Sample"].isin(excluded_samples)]
    a, b = fit_j_curve(data["SWN"], data["J"])
    rmse = round(root_mean_squared_error(data["J"], a * data["SWN"] ** b), 4)
    j_params[sample] = (a, b, rmse)

In [ ]:
# Convert j_params dictionary to DataFrame
j_params_df = (
    pd.DataFrame.from_dict(j_params, orient="index", columns=["a", "b", "rmse"])
    .reset_index()
    .rename(columns={"index": "ROCK_FLAG"})
)
j_params_df.to_json(
    rf"data\04_project\{project_name}\outputs\j_params.json", orient="index"
)

# Merge filtered_data with j_params_df
merged_data = j_params_df.merge(
    filtered_data[["Sample", "ROCK_FLAG"]].drop_duplicates(), how="left", on="ROCK_FLAG"
)

# Group by ROCK_FLAG and sort by the first value of the parameters
sorted_samples = (
    merged_data.groupby("ROCK_FLAG")
    .apply(lambda x: x.sort_values(by="rmse"))
    .reset_index(drop=True)
)

# Select samples considering values of 'a'
selected_samples = {}
hist_rmse_values = set([0])
for rock_flag in sorted_samples["ROCK_FLAG"].unique():
    for _, row in (
        sorted_samples[sorted_samples["ROCK_FLAG"] == rock_flag]
        .sort_values(by="rmse")
        .iterrows()
    ):
        if row["rmse"] not in hist_rmse_values or all(
            row["rmse"] > val for val in hist_rmse_values
        ):
            selected_samples[rock_flag] = row
            hist_rmse_values.add(row["rmse"])
            break
        else:
            selected_samples[rock_flag] = row
selected_samples = pd.DataFrame(selected_samples).T
selected_samples.to_json(
    rf"data\04_project\{project_name}\outputs\rt_j_params.json", orient="index"
)

In [ ]:
import json

with open(rf"data\04_project\{project_name}\outputs\rt_j_params.json", "r") as file:
    mapped_fzi_params = json.load(file)
mapped_fzi_params

In [ ]:
from ipywidgets import interact, widgets

rt_widget = widgets.Dropdown(
    options=sorted(core_data["ROCK_FLAG"].unique()), description="Rock Type:"
)


@interact(rt=rt_widget)
def param(rt):
    str_rt = str(rt)
    params = next(item for item in mapped_fzi_params if item["ROCK_FLAG"] == str_rt)
    a, b = params["a"], params["b"]
    data = core_data[core_data["ROCK_FLAG"] == rt]
    data = data[~data["Sample"].isin(excluded_samples)]

    j_xplot(
        data["SWN"],
        data["J"],
        a=a,
        b=b,
        core_group=data["Sample"],
        log_log=True,
        ylim=(0, 0.5),
        label=f"Rock Type {rt}: a:{a}, b:{b}",
    )

In [ ]:
# Get unique rock flags
unique_rock_flags = sorted(core_data["ROCK_FLAG"].unique())

# Create subplots
fig, axes = plt.subplots(nrows=4, ncols=3, figsize=(20, 25))
axes = axes.flatten()

# Plot j_xplot for each rock flag
for i in range(len(unique_rock_flags)):
    int_rock = i + 1
    rock = str(float(i + 1))
    ax = axes[i]
    data = core_data[core_data["ROCK_FLAG"] == int_rock]
    params = next(item for item in mapped_fzi_params if item["ROCK_FLAG"] == rock)
    a, b = params["a"], params["b"]
    ax = j_xplot(
        data["SWN"],
        data["J"],
        a=a,
        b=b,
        core_group=data["Sample"],
        label=f"a:{a}\nb:{b}",
        ax=ax,
        ylim=(0, 0.5),
    )
    ax.set_title(f"RRT {int_rock}")
    ax.legend()
    ax.grid(True)

# Hide any unused subplots
for j in range(i + 1, len(axes)):
    fig.delaxes(axes[j])

fig.set_facecolor("aliceblue")
plt.tight_layout()
plt.show()


In [ ]:
# Get unique rock flags
unique_rock_flags = sorted(core_data["ROCK_FLAG"].unique())

# Create subplots
fig, axes = plt.subplots(nrows=4, ncols=3, figsize=(20, 25))
axes = axes.flatten()

# Plot j_xplot for each rock flag
for i in range(11):
    int_rock = i + 1
    rock = str(float(i + 1))
    ax = axes[i]
    data = core_data[core_data["ROCK_FLAG"] == int_rock]
    params = next(item for item in mapped_fzi_params if item["ROCK_FLAG"] == rock)
    a, b = params["a"], params["b"]
    ax = j_xplot(
        data["SWN"],
        data["J"],
        log_log=True,
        a=a,
        b=b,
        core_group=data["Sample"],
        label=f"a:{a}\nb:{b}",
        ax=ax,
        ylim=(0, 0.5),
    )
    ax.set_title(f"RRT {int_rock}")
    ax.legend()
    ax.grid(True)

# Hide any unused subplots
for j in range(i + 1, len(axes)):
    fig.delaxes(axes[j])

fig.set_facecolor("aliceblue")
plt.tight_layout()
plt.show()


In [ ]:
import numpy as np

from quick_pp.utils import inv_power_law_func

# Plot mapped_fzi_params on the same j_xplot
for rock in core_data.ROCK_FLAG.unique():
    # Extract values from the dictionary 'd'
    params = next(item for item in mapped_fzi_params if item["ROCK_FLAG"] == rock)
    a, b = params["a"], params["b"]
    csw = np.geomspace(0.01, 1.0, 50)
    plt.plot(
        csw,
        inv_power_law_func(csw, a, b),
        label=f"RRT{rock_flag}, Sample {sample}, a:{a}, b:{b}",
        linestyle="dashed",
    )
plt.title("J Curve vs SW")
plt.xlabel("SW (v/v)")
plt.ylabel("J (unitless)")
plt.xlim(0, 1)
plt.ylim(0, 0.2)
plt.legend()

#### Estimate Free Water Level (FWL)

In [ ]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer

from quick_pp.rock_type import calc_fzi_perm

imp_mean = SimpleImputer(missing_values=np.nan, strategy="mean")

focused_well = "HW-6"
well_data = all_data[all_data.WELL_NAME == focused_well].copy()

well_data["LOG_RT"] = np.log10(well_data["RT"])
well_data["NDI"] = (2.95 - well_data["RHOB"]) / 1.95
well_data["GRN"] = MinMaxScaler().fit_transform(well_data[["GR"]])

# Predict ROCK_FLAG
input_features = ["GR", "NPHI", "RHOB", "NDI", "RT", "LOG_RT", "GRN"]
with open(r"data\04_project\MOCK_carbonate\outputs\fzi_rt_model.qppm", "rb") as file:
    fzi_rt_model = pickle.load(file)
well_data["ROCK_FLAG"] = fzi_rt_model.predict(well_data[input_features])

# Predict PERM
temp_df = well_data.copy()
temp_df["ROCK_PRED"] = well_data["ROCK_FLAG"]
input_features = ["GR", "NPHI", "RHOB", "NDI", "RT", "LOG_RT", "GRN", "ROCK_PRED"]
temp_df[input_features] = imp_mean.fit_transform(temp_df[input_features])
with open(r"data\04_project\MOCK_carbonate\outputs\fzi_model.qppm", "rb") as file:
    fzi_model = pickle.load(file)
fzi_ml = 10 ** (fzi_model.predict(temp_df[input_features]))
well_data["PERM"] = calc_fzi_perm(fzi_ml, well_data["PHIT"])


In [ ]:
from ipywidgets import interactive, widgets
from quick_pp.core_analysis import sw_shf_leverett_j

# Debug water saturation
water_salinity = 160e3
m = 1.9

ift = 32
theta = 30
ghc = 0.837
gw = 1.135
fwl = 8550

fwl = widgets.FloatSlider(value=fwl, min=fwl / 1.1, max=fwl * 1.1, step=1)


def plot(fwl):
    param_map = {item["ROCK_FLAG"]: item for item in mapped_fzi_params}

    # Map the ROCK_FLAG in well_data to the corresponding 'a' and 'b' values.
    a = well_data["ROCK_FLAG"].map(lambda x: param_map.get(x, {}).get("a"))
    b = well_data["ROCK_FLAG"].map(lambda x: param_map.get(x, {}).get("b"))
    shf = sw_shf_leverett_j(
        well_data["PERM"],
        well_data["PHIT"],
        well_data["DEPTH"],
        gw=gw,
        ghc=ghc,
        fwl=fwl,
        ift=ift,
        theta=theta,
        a=a,
        b=b,
    )

    temp_grad = estimate_temperature_gradient(well_data["DEPTH"], "imperial")
    rw = estimate_rw_temperature_salinity(temp_grad, water_salinity)

    swt = archie_saturation(well_data["RT"], rw, well_data["PHIT"], m=m)
    swt = swt.clip(0, 1)

    bvo = well_data["PHIT"] * (1 - swt)
    bvoh = well_data["PHIT"] * (1 - shf)

    plt.figure(figsize=(20, 1))
    plt.title(f"Water Saturation for {focused_well}")
    plt.plot(well_data["DEPTH"], swt, label="SWT")
    plt.plot(well_data["DEPTH"], well_data["SW"], label="SW")
    plt.plot(well_data["DEPTH"], shf, label="SHF")
    plt.plot(well_data["DEPTH"], np.ones(len(well_data)), color="black", linestyle="--")
    plt.ylim(0, 1.2)
    plt.legend()

    plt.figure(figsize=(20, 1))
    plt.plot(well_data["DEPTH"], bvo, label="BVO")
    plt.plot(well_data["DEPTH"], bvoh, label=r"$BVO_{SHF}$")
    plt.ylim(0, 0.5)
    plt.legend()

    plt.figure(figsize=(20, 1))
    plt.plot(well_data["DEPTH"], well_data["ROCK_FLAG"], label="Rock Flag")
    plt.legend()


interactive_plot = interactive(plot, fwl=fwl)
output = interactive_plot.children[-1]
output.layout.height = "350px"
interactive_plot

***
# Plot the results

In [ ]:
from quick_pp.plotter.plotter import plotly_log

fwl = 8575
param_map = {item["ROCK_FLAG"]: item for item in mapped_fzi_params}
# Map the ROCK_FLAG in well_data to the corresponding 'a' and 'b' values.
a = well_data["ROCK_FLAG"].map(lambda x: param_map.get(x, {}).get("a"))
b = well_data["ROCK_FLAG"].map(lambda x: param_map.get(x, {}).get("b"))
shf = sw_shf_leverett_j(
    well_data["PERM"],
    well_data["PHIT"],
    well_data["DEPTH"],
    gw=gw,
    ghc=ghc,
    fwl=fwl,
    ift=ift,
    theta=theta,
    a=a,
    b=b,
)

# Plot individual results
well_data["SWT"] = swt
well_data["SHF"] = shf
fig = plotly_log(well_data, "ft")
fig.show(config=dict(scrollZoom=True))

In [ ]:
# # Save the well data
# project.update_data(well_data)
# project.save()